In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys

for directory in ["agent_clients", "src", "utils"]:
    sys.path.append(f"../{directory}")

from currency_exchange_client import create_client
from ping_client import ping_client_service
from setup import setup_paths
setup_paths()

In [9]:
CURRENCY_SERVER_URL = "http://localhost:8001"

## Test pinging the client

In [11]:
client = await create_client(url=CURRENCY_SERVER_URL)

INFO:httpx:HTTP Request: GET http://localhost:8001/.well-known/agent.json "HTTP/1.1 200 OK"


In [12]:
client

In [13]:
client.url

'http://localhost:8001/currency'

In [16]:
from common.client.card_resolver import A2ACardResolver

resolver = A2ACardResolver(CURRENCY_SERVER_URL)
card = resolver.get_agent_card()

INFO:httpx:HTTP Request: GET http://localhost:8001/.well-known/agent.json "HTTP/1.1 200 OK"


In [20]:
card.model_dump()

{'name': 'Currency conversion agent',
 'description': 'Returns the exchange rate for the currencies of interest',
 'url': 'http://localhost:8001/currency',
 'provider': None,
 'version': '1.0.0',
 'documentationUrl': None,
 'capabilities': {'streaming': False,
  'pushNotifications': False,
  'stateTransitionHistory': False},
 'authentication': None,
 'defaultInputModes': ['text', 'text/plain'],
 'defaultOutputModes': ['text', 'text/plain'],
 'skills': [{'id': 'currency_conversion_agent',
   'name': 'Currency conversion agent',
   'description': 'Returns the exchange rate for the currencies of interest',
   'tags': ['currency_exchange'],
   'examples': ['What is the exchange rate from SGD to EUR'],
   'inputModes': None,
   'outputModes': None}]}

In [6]:
from uuid import uuid4

task_id = f"echo-task-{uuid4().hex}"
session_id = f"session-{uuid4().hex}"

In [7]:
question = "What is the exchange rate from sgd to usd?"

In [8]:
response = await ping_client_service(
    client = client,
    task_id = task_id,
    session_id = session_id,
    text = question
)

INFO:ping_client:Sending task echo-task-0e63b4df927a4e9ab3da56ca6c378b42 to server...


INFO:httpx:HTTP Request: POST http://localhost:8001/currency "HTTP/1.1 200 OK"
INFO:ping_client:Task echo-task-0e63b4df927a4e9ab3da56ca6c378b42 completed with state: TaskState.COMPLETED


In [11]:
print(response)

id='echo-task-0e63b4df927a4e9ab3da56ca6c378b42' sessionId='session-a19d00b6c06e472ea695ef628b359747' status=TaskStatus(state=<TaskState.COMPLETED: 'completed'>, message=None, timestamp=datetime.datetime(2025, 4, 21, 16, 20, 15, 984417)) artifacts=[Artifact(name=None, description=None, parts=[TextPart(type='text', text='The exchange rate from Singapore Dollar (SGD) to US Dollar (USD) is approximately 0.76216 USD for 1 SGD.', metadata=None)], metadata=None, index=0, append=False, lastChunk=None)] history=[] metadata=None


In [17]:
from IPython.display import display, Markdown

display(Markdown(response.artifacts[0].parts[0].text))

The exchange rate from Singapore Dollar (SGD) to US Dollar (USD) is approximately 0.76216 USD for 1 SGD.